In [ ]:
# Change the directory path
%cd ../featureSelection/

In [ ]:
import pandas as pd
import h2o
h2o.init()
from h2o.automl import H2OAutoML
from sklearn import metrics

In [ ]:
X_train = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\UnivariateFeatures.csv'))
y_train = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\TrainLabels.csv'))
X_test = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\UnivariateFeaturesTest.csv'))
y_test = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\TestLabels.csv')) 


In [ ]:
# preparing the train and test data sets
# now convert tweet vecs and labels to a pandas dataframe and back to h2o dataframe
# data = job_titles[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])
train = X_train.cbind(y_train)
test = X_test.cbind(y_test)


In [ ]:
# more on data prep
x = train.columns         # x: A list/vector of predictor column names or indexes. 
                          # This argument only needs to be specified if the user wants to exclude columns from the 
                          # set of predictors. If all columns (other than the response) should be used in prediction, 
                          # then this does not need to be set.

y = "labels"              # This argument is the name (or index) of the response column
x.remove(y)

# need to set train and test
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
# now the AUTO-ML piece comes in
aml = H2OAutoML(max_models=10) #max_models=10 or 20?, seed?
aml.train(x=x, y=y, training_frame=train)

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
# The leader model is stored here
aml.leader

In [ ]:
preds = aml.predict(test)
print(preds)

In [ ]:
var = preds["predict"].cbind(test[y])
print(var)

In [ ]:
# convert to pandas dataframe
y_test = h2o.as_list(test[y], use_pandas=True)
y_pred = h2o.as_list(preds["predict"])
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))